In [4]:
import pandas as pd 
import numpy as np 
from pathlib import Path 
import os 

In [2]:
companies_dict = {
    "UBSFY": 0.1, 
    "EA": 0.6, 
    "TTWO": 0.1,
    "ATVI": 0.1
}

In [19]:
bot_path = Path(os.path.abspath("")).parents[0] / "trading_bots"
save_path = Path(os.path.abspath("")).parents[0] / "paper"

In [6]:
data = {}
for ticker, model_version in companies_dict.items():
    data[ticker] = pd.read_csv(bot_path / f"{ticker}_{model_version}_hyperopt.csv")

In [8]:
df = pd.DataFrame()
for ticker, ticker_df in data.items(): 
    tmp = ticker_df
    tmp['ticker'] = ticker
    df = df.append(tmp)

In [12]:
df.groupby('ticker')['end_balance'].max()

ticker
ATVI       5158.735721
EA         2087.396933
TTWO     143115.038074
UBSFY      1725.066076
Name: end_balance, dtype: float64

In [11]:
df

,end_balance,top_cut_off,down_cut_off,if_short,ticker
0,1201.921776,0.0,0.0,True,UBSFY
1,1213.136911,0.0,0.0,False,UBSFY
2,1261.300968,0.0,0.2,True,UBSFY
3,1220.728518,0.0,0.2,False,UBSFY
4,1175.654551,0.0,0.4,True,UBSFY
...,...,...,...,...,...
795,1000.000000,3.8,3.4,False,ATVI
796,1051.744802,3.8,3.6,True,ATVI
797,1000.000000,3.8,3.6,False,ATVI
798,1047.459431,3.8,3.8,True,ATVI


In [14]:
idx = df.groupby(['ticker'])['end_balance'].transform(max) == df['end_balance']

In [17]:
df[idx][['ticker', "top_cut_off", "down_cut_off", "if_short"]]

,ticker,top_cut_off,down_cut_off,if_short
374,UBSFY,1.8,1.4,True
526,EA,2.6,0.6,True
324,TTWO,1.6,0.4,True
22,ATVI,0.0,2.2,True


In [20]:
with open(save_path / "signals_hpars_results.tex", "w") as f:
    f.write(
         df[idx][['ticker', "top_cut_off", "down_cut_off", "if_short"]]
        .to_latex(caption="Best strategy hyper parameters")
    )

In [26]:
tickers = ['UBSFY', 'EA', 'TTWO', 'ATVI']
buy_and_hold = [631.839, 935.412, 944.500, 660.481]
results1 = [878.008, 1006.768, 876.744, 631.715]
results2 = [1258.925, 1079.715, 870.907, 1110.533]

In [27]:
results = pd.DataFrame(
    {
        "ticker": tickers, 
        "Buy and Hold": buy_and_hold, 
        "Proposed strategy with parameters found in training set hyperparameter optymalization": results1, 
        "Proposed strategy with arbitrary chosen parameters": results2
    }
)

In [28]:
results

,ticker,Buy and Hold,Proposed strategy with parameters found in training set hyperparameter optymalization,Proposed strategy with arbitrary chosen parameters
0,UBSFY,631.839,878.008,1258.925
1,EA,935.412,1006.768,1079.715
2,TTWO,944.500,876.744,870.907
3,ATVI,660.481,631.715,1110.533


In [ ]:
with open(save_path / "strategy_results.tex", "w") as f:
    f.write(
         results
        .to_latex(caption="Diffrent strategies results")
    )